In [1]:
# 1) 데이터 수집

# 2 필수 라이브러리 호출 
    # 넘파이 판다스 사이킷런 모델셀렉션 => train_test_split, KFold
    # 알고리즘 (DecisionTreeClassfier) => 한 변수를 기준으로 각 y 클래스를 구분 
    # 평가지표 Accuracy_score(confusion_matrix{오차행렬, 혼동행렬})

# 3 데이터 정보 확인 
    # 결측치
    # 데이터 object, int, float
    # 시각화 - EDA
    
# 4 데이터 전처리
    # 결측값 대체 
    # 문자를 숫자로 변환
    # 식별자(ID, Index, 주민번호)제거
    
# 5 호출된 알고리즘 => fit, pred (모델링)

# 6 평가지표에 따라 결과지표

# 7 하이퍼파라미터 튜닝 => 6번의 결과지표를 좀 더 좋게..? 할 수 있는 방법 모색 (그리드서치CV) 

# 8 K-Fold 오버피팅 여부 체크 

In [2]:
# 필수 라이브러리 호출
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

In [21]:
# 데이터 정의(load)

st_default_df = pd.read_csv('./st_default.csv')
st_default_df1 = st_default_df.iloc[:,1:] # iloc 는 위치기반 인덱싱 (poitional indexing) 
st_default_df1

In [4]:
# train_test_split 하기 전에 문자형 데이터들 변경 - 1)레이블 vs 2) one-hot 
# 그런데 y_label의 경우 default를 예측하는 값이므로 이 값은 label
# 나머지 student는 get_dummies()

X_data = st_default_df1.drop('default', axis=1)
y_target = st_default_df1['default']

In [5]:
# EDA를 통한 데이터 확인

In [6]:
X_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   student  10000 non-null  object 
 1   balance  10000 non-null  float64
 2   income   10000 non-null  float64
dtypes: float64(2), object(1)
memory usage: 234.5+ KB


In [7]:
X_data.describe()

,balance,income
count,10000.000000,10000.000000
mean,835.374886,33516.981876
std,483.714985,13336.639563
min,0.000000,771.967729
25%,481.731105,21340.462905
50%,823.636973,34552.644800
75%,1166.308387,43807.729275
max,2654.322576,73554.233500


In [8]:
# 크기 확인 
print('X_data의 크기:', X_data.shape[0])
print('y_target의 크기:', y_target.shape[0])

X_data의 크기: 10000
y_target의 크기: 10000


In [9]:
# 문자 숫자로 변환 => 1. 컬럼 없애도 ok
                 # 2. 문자 -> 원-핫 인코딩 // laber Encoding

In [10]:
X_data_df = pd.get_dummies(X_data) # 원-핫 인코딩

In [11]:
# y_값에 label encoding

from sklearn.preprocessing import LabelEncoder

le_encode = LabelEncoder()
y_target_scaled = le_encode.fit_transform(y_target)

# imbalanced data => y타겟값을 확인 (쏠림데이터)
pd.Series(y_target_scaled).value_counts()

0    9667
1     333
dtype: int64

In [12]:
# 데이터 분할 - train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_data_df, y_target_scaled, stratify=y_target_scaled) 

In [13]:
# 의사결정나무 적용

dt_clf = DecisionTreeClassifier()
dt_clf.fit(X_train, y_train) # 학습

# 예측
y_pred = dt_clf.predict(X_test)

In [14]:
# 평가지표

print(f'정확도 : {accuracy_score(y_pred,y_test)}') 

정확도 : 0.9604


#### 배우신 분들을 위한 내용

In [15]:
# 1. 교차검증을 수행 (cross_val_score)
from sklearn.model_selection import cross_val_score

scores = cross_val_score(dt_clf, X_data_df, y_target_scaled, scoring='accuracy', cv=10) 
print(f'교차 검증별 정확도:{np.round(scores, 4)}')
print(f'평균 검증별 정확도:{np.round(np.mean(scores), 4)}')

교차 검증별 정확도:[0.953 0.959 0.947 0.956 0.957 0.957 0.957 0.958 0.957 0.954]
평균 검증별 정확도:0.9555


In [16]:
# 2. 하이퍼파라미터 튜닝을 수행
from sklearn.model_selection import GridSearchCV

parameters = {'max_depth':[3,4,5,6,7,10],
              'min_samples_split':[2,3,5,10],}
grid_dtree = GridSearchCV(dt_clf, param_grid=parameters, cv=10, refit=True)
grid_dtree.fit(X_train,y_train)

GridSearchCV(cv=10, estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': [3, 4, 5, 6, 7, 10],
                         'min_samples_split': [2, 3, 5, 10]})

In [17]:
estimator = grid_dtree.best_estimator_

pred_grid = estimator.predict(X_test)
print(f'test datasets accuracy:{accuracy_score(y_test,pred_grid)}')

test datasets accuracy:0.974


In [18]:
y_target.value_counts()

No     9667
Yes     333
Name: default, dtype: int64

In [19]:
# 1. 왜 데이터의 해당하는 정확도 값이 높게 나왔을까요?
# 2. 데이터는 어떠한 특징을 지니고 있나요?
# 3. 데이터에는 어떠한 문제점이 있나요?
# 4. 그렇다면 이러한 문제점을 해결하기 위한 방안은 무엇이 있을까요?
    # 1. over sampling, under sampling
    # 2. Data augementation => 이미지 분석에서 자주 쓰임
    # 4. get more data
    # 5. change our metrics - precision, recall, f1-score, roc_auc

In [20]:
# end of file